# Abordagem 3

Usando a abordagem 3 para gerar templates com foco em templates positivos e negativos. Uma possível aplicação seria testar a capacidade linguística *Vocabullary* com o teste **MFT**.

As etapas desta abordagem são:

1. Quebrar a instância em sentenças
2. Classificar as sentenças usando o *Oráculo*
3. Filtrar as sentenças classificadas de forma unânime
4. Filtrar as sentenças com alta confiança nas predições
5. Rankear as palavras de cada sentença
6. Filtrar sentenças com palavras relevantes (verbos ou adjetivos) bem rankeadas
7. Filtrar sentenças com alta confiança na predição das palavras relevantes 
8. Substituir as palavras relevantes por máscaras

In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../../')

## Carregando o dataset, o modelo alvo e os modelos auxiliares

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

movie_reviews_rt_df = pd.read_csv('./data/data-rt-100samples.csv')
movie_reviews_rt_df.head(5)

,label,text,words
0,1,allen's underestimated charm delivers more goodies than lumps of coal .,11
1,0,skip the film and buy the philip glass soundtrack cd .,11
2,0,involving at times but lapses quite casually into the absurd .,11
3,0,while hoffman's performance is great the subject matter goes nowhere .,11
4,1,a flick about our infantilized culture that isn't entirely infantile .,11


In [3]:
import re
import numpy as np
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def pre_proccess(text):
    text = text.lower()
    text = re.sub('["\',!-.:-@0-9/]()', ' ', text)
    return text

# Wrapper to adapt output format
class SentimentAnalisysModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __predict(self, text_input):
        text_preprocessed = pre_proccess(text_input)
        tokenized = self.tokenizer(text_preprocessed, padding=True, truncation=True, max_length=512, 
                                    add_special_tokens = True, return_tensors="pt")
        
        tensor_logits = self.model(**tokenized)
        prob = softmax(tensor_logits[0]).detach().numpy()
        pred = np.argmax(prob)
        
        return pred, prob
    
    def predict_label(self, text_inputs):
        return self.predict(text_inputs)[0]
        
    def predict_proba(self, text_inputs):
        return self.predict(text_inputs)[1]
        
    def predict(self, text_inputs):
        if isinstance(text_inputs, str):
            text_inputs = [text_inputs]
        
        preds = []
        probs = []

        for text_input in text_inputs:
            pred, prob = self.__predict(text_input)
            preds.append(pred)
            probs.append(prob[0])

        return np.array(preds), np.array(probs) # ([0, 1], [[0.99, 0.01], [0.03, 0.97]])

# Auxiliar function to load and wrap a model from Hugging Face
def load_model(model_name):
    print(f'Loading model {model_name}...')
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return SentimentAnalisysModelWrapper(model, tokenizer)

# Hugging Face hosted model names 
movie_reviews_models = {
    'bert': 'textattack/bert-base-uncased-rotten-tomatoes', 
    'albert': 'textattack/albert-base-v2-rotten-tomatoes', 
    'distilbert': 'textattack/distilbert-base-uncased-rotten-tomatoes', 
    'roberta': 'textattack/roberta-base-rotten-tomatoes', 
    'xlnet': 'textattack/xlnet-base-cased-rotten-tomatoes'
}

In [4]:
m1 = load_model(movie_reviews_models['albert'])
m2 = load_model(movie_reviews_models['distilbert'])
m3 = load_model(movie_reviews_models['roberta'])
m4 = load_model(movie_reviews_models['xlnet'])

# Models to be used as oracle
models = [m1, m2, m3, m4]
# Target model
model = load_model(movie_reviews_models['bert'])

Loading model textattack/albert-base-v2-rotten-tomatoes...
Loading model textattack/distilbert-base-uncased-rotten-tomatoes...
Loading model textattack/roberta-base-rotten-tomatoes...


Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model textattack/xlnet-base-cased-rotten-tomatoes...
Loading model textattack/bert-base-uncased-rotten-tomatoes...


# Gerando os templates
O método de rankeamento das palavras usado no PosNegTemplateGenerator é o Replace-1 Score

In [5]:
from template_generator.tasks.sentiment_analisys import PosNegTemplateGeneratorApp3

tg = PosNegTemplateGeneratorApp3(model, models)

### Número inicial de instâncias: 5

In [6]:
# Sampling instances
np.random.seed(220)
n_instances = 5
df_sampled = movie_reviews_rt_df.sample(n_instances)

instances = [x for x in df_sampled['text'].values]

In [7]:
templates = tg.generate_templates(instances,  n_masks=2, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 6 sentences were generated.
Predicting inputs...


<ipython-input-3-263ca9d23a33>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Sentence predictions done.
Filtering instances classified unanimously...
:: 3 sentences remaining.
Filtering instances by classification score greater than 0.8
:: 2 sentences remaining.
Ranking words using Replace-1 Score...
:: Word ranking done.
4
Filtering instances by relevant words...
['ADJ']
{word: of, index: 17, tag: ADP, rank_score: -0.00011205673217773438}
{word: wonders, index: 16, tag: NOUN, rank_score: -7.635354995727539e-05}
{word: elegantly, index: 0, tag: ADV, rank_score: -5.036592483520508e-05}
{word: detailed, index: 15, tag: ADJ, rank_score: -1.6689300537109375e-05}
 
['ADJ']
{word: refreshingly, index: 10, tag: ADV, rank_score: 0.0010156035423278809}
{word: no, index: 1, tag: DET, rank_score: 0.000680387020111084}
{word: by, index: 0, tag: ADP, rank_score: 0.0005499720573425293}
{word: forthright, index: 11, tag: ADJ, rank_score: -0.0003120899200439453}
 
:: 0 sentences remaining.
Filtering instances by relevant words classification score greater than 0.8
:: 0 sent

#### Tempo de execução para 5 instâncias: 6.1s

In [8]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text


In [9]:
tg.lexicons

{'pos_adj': [], 'neg_adj': []}

### Número inicial de instâncias: 100

In [10]:
# Using all 100 instances
instances = [x for x in movie_reviews_rt_df['text'].values]

In [11]:
tg = PosNegTemplateGeneratorApp3(model, models)
templates = tg.generate_templates(instances,  n_masks=2, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 134 sentences were generated.
Predicting inputs...


<ipython-input-3-263ca9d23a33>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Sentence predictions done.
Filtering instances classified unanimously...
:: 96 sentences remaining.
Filtering instances by classification score greater than 0.8
:: 87 sentences remaining.
Ranking words using Replace-1 Score...
:: Word ranking done.
4
Filtering instances by relevant words...
['ADJ']
{word: skip, index: 0, tag: VERB, rank_score: -0.3447835445404053}
{word: and, index: 3, tag: CONJ, rank_score: -0.012676358222961426}
{word: philip, index: 6, tag: NOUN, rank_score: 0.0009868144989013672}
{word: the, index: 1, tag: DET, rank_score: 0.00048273801803588867}
 
['ADJ']
{word: lapses, index: 4, tag: VERB, rank_score: -0.4460386037826538}
{word: into, index: 7, tag: ADP, rank_score: -0.0017092227935791016}
{word: times, index: 2, tag: NOUN, rank_score: 0.0014830231666564941}
{word: the, index: 8, tag: DET, rank_score: 0.001171112060546875}
 
['ADJ']
{word: criminally, index: 9, tag: ADV, rank_score: -0.9855519533157349}
{word: one, index: 8, tag: NUM, rank_score: -0.6671131849

#### Tempo de execução para 100 instâncias: 4m 22.7s

In [12]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text
0,1,intelligent caustic take on a great writer and dubious human being .,{mask} caustic take on a great writer and dubious {mask} being .,{pos_adj} caustic take on a great writer and dubious {neg_adj} being .
1,0,a long dull procession of despair set to cello music culled from a minimalist funeral .,a long {mask} procession of despair set to cello music culled from a {mask} funeral .,a long {neg_adj} procession of despair set to cello music culled from a {neg_adj} funeral .
2,0,an empty exercise a florid but ultimately vapid crime melodrama with lots of surface flash but little emotional resonance .,an empty exercise a florid but ultimately {mask} crime melodrama with lots of surface flash but {mask} emotional resonance .,an empty exercise a florid but ultimately {neg_adj} crime melodrama with lots of surface flash but {neg_adj} emotional resonance .
3,0,hugh grant's act is so consuming that sometimes it's difficult to tell who the other actors in the movie are .,hugh grant 's act is so {mask} that sometimes it 's {mask} to tell who the other actors in the movie are .,hugh grant 's act is so {neg_adj} that sometimes it 's {neg_adj} to tell who the other actors in the movie are .
4,1,more concerned with overall feelings broader ideas and open-ended questions than concrete story and definitive answers soderbergh's solaris is a gorgeous and deceptively minimalist cinematic tone poem .,more concerned with overall feelings broader ideas and open-ended questions than concrete story and definitive answers soderbergh 's solaris is a {mask} and {mask} minimalist cinematic tone poem .,more concerned with overall feelings broader ideas and open-ended questions than concrete story and definitive answers soderbergh 's solaris is a {pos_adj} and {pos_adj} minimalist cinematic tone poem .
5,0,as well-acted and well-intentioned as all or nothing is however the film comes perilously close to being too bleak too pessimistic and too unflinching for its own good .,as well-acted and well-intentioned as all or nothing is however the film comes perilously close to being too bleak too {mask} and too {mask} for its own good .,as well-acted and well-intentioned as all or nothing is however the film comes perilously close to being too bleak too {neg_adj} and too {pos_adj} for its own good .
6,0,a dreary incoherent self-indulgent mess of a movie in which a bunch of pompous windbags drone on inanely for two hours .,a dreary incoherent {mask} mess of a movie in which a bunch of {mask} windbags drone on inanely for two hours .,a dreary incoherent {neg_adj} mess of a movie in which a bunch of {neg_adj} windbags drone on inanely for two hours .
7,0,is an inexpressible and drab wannabe looking for that exact niche .,is an {mask} and {mask} wannabe looking for that exact niche .,is an {neg_adj} and {neg_adj} wannabe looking for that exact niche .


In [13]:
tg.lexicons

{'pos_adj': ['intelligent', 'gorgeous', 'unflinching', 'deceptively'],
 'neg_adj': ['little',
  'human',
  'self-indulgent',
  'pessimistic',
  'consuming',
  'drab',
  'pompous',
  'difficult',
  'minimalist',
  'inexpressible',
  'dull',
  'vapid']}

## Checklist

In [14]:
from checklist.editor import Editor
from checklist.test_suite import TestSuite
from checklist.test_types import MFT

In [15]:
lexicons = tg.lexicons
templates = tg.template_texts
masked = tg.masked_texts
labels = [sent.prediction.label for sent in tg.sentences]

editor = Editor()
editor.add_lexicon('pos_adj', lexicons['pos_adj'])
editor.add_lexicon('neg_adj', lexicons['neg_adj'])

suite = TestSuite()

In [16]:
for template, label, i in zip(templates, labels, range(len(templates))):
    t = editor.template(template, remove_duplicates=True, labels=int(label))

    suite.add(MFT(
        data=t.data,
        labels=label,
        capability="Vocabullary", 
        name=f"Test: MFT with vocabullary - template{i+1}",
        description="Checking if the model can handle vocabullary"))

In [17]:
suite.run(model.predict, overwrite=True)

Running Test: MFT with vocabullary - template1
Predicting 48 examples


<ipython-input-3-263ca9d23a33>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Running Test: MFT with vocabullary - template2
Predicting 12 examples
Running Test: MFT with vocabullary - template3
Predicting 12 examples
Running Test: MFT with vocabullary - template4
Predicting 12 examples
Running Test: MFT with vocabullary - template5
Predicting 4 examples
Running Test: MFT with vocabullary - template6
Predicting 48 examples
Running Test: MFT with vocabullary - template7
Predicting 12 examples
Running Test: MFT with vocabullary - template8
Predicting 12 examples


In [18]:
suite.summary()

Vocabullary

Test: MFT with vocabullary - template1
Test cases:      48
Fails (rate):    13 (27.1%)

Example fails:
0.0 deceptively caustic take on a great writer and dubious consuming being .
----
0.0 deceptively caustic take on a great writer and dubious dull being .
----
0.0 deceptively caustic take on a great writer and dubious pessimistic being .
----


Test: MFT with vocabullary - template2
Test cases:      12
Fails (rate):    0 (0.0%)


Test: MFT with vocabullary - template3
Test cases:      12
Fails (rate):    1 (8.3%)

Example fails:
0.8 an empty exercise a florid but ultimately human crime melodrama with lots of surface flash but human emotional resonance .
----


Test: MFT with vocabullary - template4
Test cases:      12
Fails (rate):    0 (0.0%)


Test: MFT with vocabullary - template5
Test cases:      4
Fails (rate):    1 (25.0%)

Example fails:
0.0 more concerned with overall feelings broader ideas and open-ended questions than concrete story and definitive answers soderb

In [19]:
suite.save('./suites/posneg-approach3.suite')

# Carregando suite de teste

In [20]:
from checklist.test_suite import TestSuite
suite = TestSuite.from_file('./suites/posneg-approach3.suite')

suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Test: MFT with vocab…